<a href="https://colab.research.google.com/github/gianca1990/pdf_extraction/blob/main/berami.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install pymupdf
import fitz
import json
import pandas as pd

In [2]:
def extract_form_fields(pdf_path):
    """
    Extrahiert alle Formularfelder (Checkboxen und Textfelder) aus einem PDF.
    Gibt eine JSON-Ausgabe mit den Namen und Werten der Felder zurück.
    """
    doc = fitz.open(pdf_path)
    form_data = []

    for page in doc:
        for widget in page.widgets():  # Alle Formularfelder auf der Seite durchgehen
            field_name = widget.field_name  # Name des Feldes
            field_value = widget.field_value  # Wert des Feldes

            # Überprüfen, welcher Feldtyp vorliegt
            if widget.field_type == 2:  # 2 = Checkbox
                field_type = "Checkbox"
                field_value = "On" if field_value == "On" else "Off"
            elif widget.field_type == 7:  # 1 = Textfeld (Inputfeld)
                field_type = "Text Input"
            else:
                field_type = "Unknown"  # Für andere unbekannte Feldtypen

            # Formularfeld in die Liste einfügen
            form_data.append({
                "name": field_name,
                "type": field_type,
                "value": field_value if field_value else "No value"
            })

    # JSON-Ausgabe erstellen
    json_output = json.dumps({"form_fields": form_data}, indent=4)
    return json_output

In [23]:
import json
import pandas as pd

# Beispiel: json_output ist das Ergebnis von extract_form_fields()
json_output = extract_form_fields("/content/24.0730_IQ_AB_Frau_Wuhl_DV.pdf")  # PDF-Pfad
form_data_dict = json.loads(json_output)

In [19]:
# Hilfsfunktion zur Extraktion eines bestimmten Feldwertes
def get_field_value(fields, field_name):
    for field in fields:
        if field['name'] == field_name:
            return field['value']
    return None

# Erstellt den DataFrame für "berater"
berater = pd.DataFrame([{
    "ID": None,
    "NAME BERATER*IN": get_field_value(form_data_dict["form_fields"], "NAME BERATER*IN")
}])

print(berater)

     ID NAME BERATER*IN
0  None              DV


In [20]:
import json
import pandas as pd

# Beispiel: JSON bereits als Dictionary geladen
# json_output = extract_form_fields("pfad_zur_pdf.pdf")
form_data_dict = json.loads(json_output)
fields = form_data_dict["form_fields"]

# Hilfsfunktionen
def get_value(fields, name):
    for f in fields:
        if f["name"] == name:
            return f["value"]
    return None

def get_checkbox_name_if_on(fields, name):
    return name if get_value(fields, name) == "On" else None

def get_on_checkboxes(fields, relevant_names):
    return [f["name"] for f in fields if f["name"] in relevant_names and f["value"] == "On"]

def get_anrede(fields):
    if get_value(fields, "weiblich") == "On":
        return "Frau"
    elif get_value(fields, "männlich") == "On":
        return "Herr"
    elif get_value(fields, "divers") == "On":
        return "Divers"
    elif get_value(fields, "keine Angabe zum Geschlecht") == "On":
        return "Keine Angabe"
    return "Unbekannt"

def get_ort(fields):
    if get_value(fields, "Wohnsitz in Deutschland") == "On":
        return get_value(fields, "Wohnort RS in Deutschland")
    elif get_value(fields, "Wohnsitz außerhalb Deutschlands") == "On":
        return get_value(fields, "Land des aktuellen Wohnsitzes RS, wenn nicht Deutschland")
    return None

def get_wer_wird_beraten(fields):
    if get_value(fields, "RS selbst") == "On":
        return "RS selbst"
    elif get_value(fields, "soziales Umfeld") == "On":
        return "soziales Umfeld"
    return "Keine Angabe"

# Zugangswege zur Beratung (nur relevante, die "On" sind)
zugangswege_felder = [
    "Agentur für Arbeit oder Jobcenter",
    "Arbeitgeber (Unternehmen)",
    "Ausländerbehörde",
    "Bildungsdienstleister",
    "Ehrenamtliche Unterstützung",
    "eigene Öffentlichkeitsarbeit zB Projektwebseite Flyer Infoveranstaltung",
    "IQ interner Verweis",
    "IQ externe Öffentlichkeitsarbeit Internet Presse Anerkennungsportal BIBB Social",
    "persönliche Empfehlung",
    "Zentrale Servicestelle Berufsanerkennung ZSBA in Bonn"
]

# DataFrame erstellen
kunde = pd.DataFrame([{
    "ID": None,
    "Anrede": get_anrede(fields),
    "Titel": get_value(fields, "TITEL Abschluss 1 bzw. REFERENZBERUF 1"),
    "Nachname": get_value(fields, "Name RS"),
    "Vorname": get_value(fields, "Vorname RS"),
    "Strasse Hausnummer": None,
    "PLZ": None,
    "Ort": get_ort(fields),
    "Telefonnummer RS": get_value(fields, "Telefonnummer RS"),
    "E-Mail": get_value(fields, "E-Mail RS"),
    "Wer wird beraten": get_wer_wird_beraten(fields),
    "Geburtsdatum": get_value(fields, "Geburtsjahr"),
    "Geburtsland": get_value(fields, "Geburtsland RS"),
    "Einreisejahr": get_value(fields, "Einreisejahr"),
    "Lebt im Ausland": get_value(fields, "Wohnsitz außerhalb Deutschlands"),
    "Staatsangehörigkeit 1": get_value(fields, "Staatsangehörigkeit 1"),
    "Staatsangehörigkeit 2": get_value(fields, "Staatsangehörigkeit 2"),
    "Zugangswege zur Beratung": ", ".join(get_on_checkboxes(fields, zugangswege_felder)),
    "Datum der Beratung": get_value(fields, "DATUM der Beratung"),
    "Kontaktaufnahme erlaubt": get_value(fields, "telefonische Beratung"),
    "schriftliche Einwilligungserklärung": get_value(fields, "schriftliche Einwilligungserklärung liegt vor")
}])

# Ausgabe prüfen
print(kunde)

     ID Anrede                                              Titel Nachname  \
0  None   Frau  Lehramt + Bachelor in Education+ Master in Edu...     Wuhl   

  Vorname Strasse Hausnummer   PLZ        Ort Telefonnummer RS  \
0  Ariela               None  None  Frankfurt     016093455298   

               E-Mail  ... Geburtsdatum Geburtsland Einreisejahr  \
0  arielaw9@gmail.com  ...         1963      Israel         2023   

  Lebt im Ausland Staatsangehörigkeit 1 Staatsangehörigkeit 2  \
0             Off            israelisch              No value   

            Zugangswege zur Beratung Datum der Beratung  \
0  Agentur für Arbeit oder Jobcenter           30.07.24   

  Kontaktaufnahme erlaubt schriftliche Einwilligungserklärung  
0                      On                                 Off  

[1 rows x 21 columns]


In [21]:
import pandas as pd
import json

# Beispiel: JSON bereits geladen
# json_output = extract_form_fields("pfad_zur_pdf.pdf")
form_data_dict = json.loads(json_output)
fields = form_data_dict["form_fields"]

# Hilfsfunktionen
def get_value(fields, name):
    for f in fields:
        if f["name"] == name:
            return f["value"]
    return None

def get_on_values(fields, name_list):
    return [f["name"] for f in fields if f["name"] in name_list and f["value"] == "On"]

# Gruppierungen
erwerbsstatus_felder = [
    "beitragspflichtig beschäftigt",
    "geringfügig beschäftigt (Mini-Job)",
    "selbstständig"
]

erwerbstaetigkeit_felder = [
    "Erwerbstätigkeit im Ausland",
    "Erwerbstätigkeit in Deutschland",
    "keine Erwerbstätigkeit",
    "Keine Erwerbstätigkeit, jedoch in Aus Weiterbildung",
    "keine Angabe zur Erwerbstätigkeit"
]

leistungsbezug_felder = [
    "ohne Leistungsbezug",
    "ohne Leistungsbezug jedoch arbeitssuchend gemeldet",
    "mit ergänzendem SGB IILeistungsbezug",
    "mit SGB IIILeistungsbezug",
    "mit SGB IIIund SGB IILeistungsbezug",
    "mit ergänzendem Asylbewerberleistungsbezug",
    "keine Angabe Leistungsbezug"
]

aufenthaltstitel_felder = [
    "Aufenthaltserlaubnis zur Arbeitsplatzsuche 20 AufenthG",
    "Aufenthaltserlaubnis für in anderen Mitgliedsstaaten der Europäischen Union",
    "Aufenthaltsgestattung 55 Abs 1 AsylVfG",
    "Aufenthalt aus familiären Gründen 2736 AufenthG",
    "Aufenthalt aus völkerrechtlichen humanitären oder politischen Gründen 2226",
    "Aufenthalt zum Zweck der Ausbildung 16a und 17 AufenthG",
    "Aufenthalt zum Zweck der Erwerbstätigkeit 18 18a 20 21 AufenthG",
    "Aufenthalt zum Zweck einer Anpassungsqualifizierung oder einer Kenntnisprüfung",
    "Blaue Karte EU 18b Abs 2 AufenthG",
    "Duldung 60a 60b 60c 60d AufenthG",
    "Niederlassungserlaubnis 9 AufenthG",
    "Staatsbürgerin EUEWRSchweiz oder Freizügigkeit 25 1213 15 FreizügGEU 28",
    "Visum 6 AufenthG",
    "kein Aufenthaltstitel da Wohnsitz im Ausland",
    "keine Angabe Aufenthaltstatus"
]

# DataFrame erzeugen
erwerbsstatus_und_aufenthaltstatus = pd.DataFrame([{
    "ID": None,
    "Erwerbsstatus": ", ".join(get_on_values(fields, erwerbsstatus_felder)),
    "erwerbstätig?": ", ".join(get_on_values(fields, erwerbstaetigkeit_felder)),
    "Leistungsbezug": ", ".join(get_on_values(fields, leistungsbezug_felder)),
    "Aufenthaltstitel": ", ".join(get_on_values(fields, aufenthaltstitel_felder)),
    "Eingabedatum": get_value(fields, "DATUM der Beratung")
}])

# Ausgabe prüfen
print(erwerbsstatus_und_aufenthaltstatus)

     ID                  Erwerbsstatus           erwerbstätig?  \
0  None  beitragspflichtig beschäftigt  keine Erwerbstätigkeit   

        Leistungsbezug                                 Aufenthaltstitel  \
0  ohne Leistungsbezug  Aufenthalt aus familiären Gründen 2736 AufenthG   

  Eingabedatum  
0     30.07.24  


In [25]:
import pandas as pd
import json

# Angenommen: form_data_dict = json.loads(json_output)
fields = form_data_dict["form_fields"]

# Hilfsfunktionen
def get_on_values(fields, name_list):
    return [f["name"] for f in fields if f["name"] in name_list and f["value"] == "On"]

# Gruppenbezeichner
deutschkenntnisse_felder = [
    "Deutsch als MUTTERSPRACHE",
    "Deutsch als FREMDSPRACHE",
    "KEINE DEUTSCHKENNTNISSE",
    "KEINE ANGABE Deutschkenntnisse"
]

zertifikat_felder = [
    "Sprachniveau zertifiziert A1",
    "Sprachniveau zertifiziert A2",
    "Sprachniveau zertifiziert B1",
    "Sprachniveau zertifiziert B2",
    "Sprachniveau zertifiziert C1",
    "Sprachniveau zertifiziert C2",
    "keine Angabe zum Niveau des Sprachzertifikats"
]

# DataFrame erzeugen
deutschkenntnisse = pd.DataFrame([{
    "ID": None,
    "Deutschkenntnisse": ", ".join(get_on_values(fields, deutschkenntnisse_felder)),
    "Zertifikat Sprachniveau": ", ".join(get_on_values(fields, zertifikat_felder)),
    "Gespräch auf": None
}])

# Ausgabe prüfen
print(deutschkenntnisse)


     ID         Deutschkenntnisse       Zertifikat Sprachniveau Gespräch auf
0  None  Deutsch als FREMDSPRACHE  Sprachniveau zertifiziert A1         None
